<a href="https://colab.research.google.com/github/simpostor/resumeparse/blob/main/resume_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 1.2 MB 8.9 MB/s 
     |████████████████████████████████| 5.8 MB 61.1 MB/s 
     |████████████████████████████████| 7.6 MB 63.3 MB/s 
     |████████████████████████████████| 182 kB 52.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm  
import json

In [4]:
spacy.__version__

'3.4.4'

In [5]:
!git clone https://github.com/simpostor/resumeparse.git

Cloning into 'resumeparse'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [6]:
cv_data = json.load(open('/content/resumeparse/resumeparse/data/training/train_data.json'))

In [7]:
len(cv_data)

200

In [8]:
!python -m spacy init fill-config /content/resumeparse/resumeparse/data/training/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [10]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()
        
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']
    
    ents = []
    entity_indices = []
            
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue 
                    
      entity_indices = entity_indices + list(range(start, end))  
                
      try:
        span = doc.char_span(start, end,label=label,alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + " " + str(text) + "\n "
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass 
  return db
                    

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [12]:
 len(train),len(test)

(140, 60)

In [13]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 100.29it/s]


In [14]:
len(db.tokens)

60

In [16]:
!python -m spacy train /content/resumeparse/resumeparse/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-28 07:50:52,264] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-28 07:50:52,275] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-28 07:50:52,278] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-28 07:50:52,279] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 683kB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.15MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 8.09MB/s]
Downloading: 100% 501M/501M [00:05<00:00, 85.9MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS ex

In [23]:
nlp = spacy.load('/content/output/model-best')

In [37]:
doc = nlp('My name is Atharva Sankhe. I worked at Google and Microsoft I graduated in 2022 from Ram Rao Adik Insitute of Technology with a bachelors degree in computer science. SKILLS are Machine Learning and Data Science')
for ent in doc.ents:
  print(ent.label_.upper(),"  =>  ", ent.text)

NAME   =>   Atharva Sankhe
COMPANIES WORKED AT   =>   Google
COMPANIES WORKED AT   =>   Microsoft
GRADUATION YEAR   =>   2022
COLLEGE NAME   =>   Ram Rao Adik Insitute of Technology
DEGREE   =>   a bachelors degree in computer science
SKILLS   =>   Machine Learning and Data Science
